In [ ]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [ ]:
train

,Unnamed: 0,content,tag
0,32649,최근 흰머리가 많이 생겨서 걱정을 많이 있어 걱정했습니다. 매번 감사합니다.,1
1,33755,그래서 방법은 우리 스스로 예방규칙 잘 지키고 우리 스스로 면역체계를 강화,1
2,33298,코로나 19에 대해서 잘 알지도 못한 상황이었기 때문에 당연히 실수가 있었을 거라고...,1
3,33854,매우 흥미롭고 유익한 기사네요.,1
4,32825,고생 많으세요~ 진본이랑 우리나라 의료진들~ 끝까지 파이팅 합시다~~~~~~~!!!!!!,1
...,...,...,...
386,32471,확진자가 점점 줄고 있는 추세이고 앞으로 이대로만 하면 코로나 이겨날 수 있을 것 ...,1
387,34002,a,1
388,32926,잘한일,1
389,33454,코로나 19 항체치료제가 생산 계획에 있다는 소식만으로도 정말 다행이라고 생각합니다...,1


In [ ]:
word = ["코로나","당직실","처리","문자","접수","신청","요청","내용","결과","담당자","종결","확인","대한민국","공유","신고","기관","연락처","답변",
            "공개","민원사무","대한","회신","번호","다른","처분","해당","전환","서울","서울시","일산","위원","청구인","민원인","바로"]

In [ ]:
for i in ran

In [ ]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/긍정.csv', encoding = 'utf-8')

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(text, test_size=0.2,
                                                    shuffle=True, random_state=1004)

In [ ]:
train.rename(columns = {'text':'content', 'label':'tag'}, inplace = True)
train

,content,tag
0,이미 또 터졌 택시 기사 어쩌 아이 학교 불러들이 부모 나라 국민 도저히 도저히 이...,0
1,코로나 19 항체치료제가 생산 계획에 있다는 소식만으로도 정말 다행이라고 생각합니다...,1
2,서구의 글로벌 회사들의 치료제. 백신은 국적과 무관하게 모두 미 FDA 승인을 받는...,2
3,ㅎㄷㄷ,2
4,쟤내는 멀해도 안되ㅋㅋ,2
...,...,...
4715,파이팅,1
4716,서울 출장 가야 정말 가 싫,0
4717,양심은있네 빨리 백신좀 만들자 이제,2
4718,제발 요금 정산해라... 비행기 타고 온 거부터 모조리 싹 다,2


## **전처리 - train set**

In [ ]:
# 댓글 추출
sentences = train['content']

NameError: ignored

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

In [ ]:
# tag 추출
labels = train['tag'].values
labels[:10]

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 150

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

In [ ]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

In [ ]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

## **전처리 - test set**

In [ ]:
import pandas as pd
test = pd.read_csv('/content/drive/MyDrive/test_우_d_중.csv', encoding = 'utf-8')

In [ ]:
test['tag'] = test[',content,tag'].str[-2].astype(int)
test['content'] = test['content,tag'].str[:-3]
test = test.iloc[:,[2,1]]
test

KeyError: ignored

In [ ]:
test.rename(columns = {'text':'content', 'label':'tag'}, inplace = True)
test

,Unnamed: 0,index,content,tag
0,0,4683,군법 파괴자~~정답!!!,2
1,1,2979,"개신교 다 비대면 예배 하겠다고 발표했고 ,1.2월부터 온라인 예배드리고 있어요.전...",2
2,2,678,맞벌이 라 얼 집 긴급 보육 다니 막내 넘 걱정 가게 접 없,0
3,3,407,장기전 될 같 입국 금지 해외 유입 막 내수 라도 살려야 입국 제한 제발 한국 일상...,0
4,4,4086,검사 수가 적어 저 정도지 엄청날 것입니다시 한 폭탄,2
...,...,...,...,...
1176,1176,1538,이럴 때 아니 또 퍼진다,0
1177,1177,404,아 개학 얼마 안 남 어쩌 긴 연기 점점 조심 곳곳 무뎌져 끝 긴장 놓치 안 되,0
1178,1178,1878,제발 좀 자국민 좀 살려도,0
1179,1179,2017,중국 세계 사죄 피해 보상 소중 일상 다 망가졌,0


In [ ]:
# 댓글 추출
sentences = test['content']
sentences[:10]

0                                        군법 파괴자~~정답!!!
1    개신교 다 비대면 예배 하겠다고 발표했고 ,1.2월부터 온라인 예배드리고 있어요.전...
2                   맞벌이 라 얼 집 긴급 보육 다니 막내 넘 걱정 가게 접 없 
3    장기전 될 같 입국 금지 해외 유입 막 내수 라도 살려야 입국 제한 제발 한국 일상...
4                        검사 수가 적어 저 정도지 엄청날 것입니다시 한 폭탄
5                                     용인 오늘 확진 떴 끝나 하 
6    모기 전파 가능 여름 제일 위험 지도 초기 이란 퍼지 보 날씨 따뜻 전파력 떨어지 ...
7                                                 무서워 
8                어찌 보 참 안 따 운 사람 이 나라 기업 죽인 전형 인 사건 이 
9                                    이상하게 중국인만 완치?????
Name: content, dtype: object

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 군법 파괴자~~정답!!! [SEP]',
 "[CLS] 개신교 다 비대면 예배 하겠다고 발표했고 ,1.2월부터 온라인 예배드리고 있어요.전광훈(한기총)은 1989년에 개신교에서 나간 집단입니다!!!기독교 총연합회가 한국 개신교 총 연합회가 아닙니다.!부산 대면 예배 본 15퍼센트 교회들도 다 전광훈 쪽 교회들'입니다.딱히  [SEP]",
 '[CLS] 맞벌이 라 얼 집 긴급 보육 다니 막내 넘 걱정 가게 접 없  [SEP]',
 '[CLS] 장기전 될 같 입국 금지 해외 유입 막 내수 라도 살려야 입국 제한 제발 한국 일상생활 되찾 지친다  [SEP]',
 '[CLS] 검사 수가 적어 저 정도지 엄청날 것입니다시 한 폭탄 [SEP]',
 '[CLS] 용인 오늘 확진 떴 끝나 하  [SEP]',
 '[CLS] 모기 전파 가능 여름 제일 위험 지도 초기 이란 퍼지 보 날씨 따뜻 전파력 떨어지 아닌 같 제발 빨리 끝났으면 좋  [SEP]',
 '[CLS] 무서워  [SEP]',
 '[CLS] 어찌 보 참 안 따 운 사람 이 나라 기업 죽인 전형 인 사건 이  [SEP]',
 '[CLS] 이상하게 중국인만 완치????? [SEP]']

In [ ]:
# tag 추출
labels = test['tag'].values
labels

array([2, 2, 0, ..., 0, 0, 0])

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 군법 파괴자~~정답!!! [SEP]
['[CLS]', '군', '##법', '파', '##괴', '##자', '~', '~', '정', '##답', '!', '!', '!', '[SEP]']


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 150

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   8910,  33768,   9901, 118651,  13764,    198,    198,
         9670, 118775,    106,    106,    106,    102,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   8910,  33768,   9901, 118651,  13764,    198,    198,   9670,
        118775,    106,    106,    106,    102,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [ ]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

## **모델 생성**

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)
model.cuda() #model.to(device) #model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 10

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")

NameError: ignored

In [ ]:
import pandas as pd
test = pd.read_csv('/content/drive/MyDrive/test_우_d_중.csv', encoding = 'utf-8')

In [ ]:
test.rename(columns = {'text':'content', 'label':'tag'}, inplace = True)
test

,content,tag
0,군법 파괴자~~정답!!!,2
1,"개신교 다 비대면 예배 하겠다고 발표했고 ,1.2월부터 온라인 예배드리고 있어요.전...",2
2,맞벌이 라 얼 집 긴급 보육 다니 막내 넘 걱정 가게 접 없,0
3,장기전 될 같 입국 금지 해외 유입 막 내수 라도 살려야 입국 제한 제발 한국 일상...,0
4,검사 수가 적어 저 정도지 엄청날 것입니다시 한 폭탄,2
...,...,...
1176,이럴 때 아니 또 퍼진다,0
1177,아 개학 얼마 안 남 어쩌 긴 연기 점점 조심 곳곳 무뎌져 끝 긴장 놓치 안 되,0
1178,제발 좀 자국민 좀 살려도,0
1179,중국 세계 사죄 피해 보상 소중 일상 다 망가졌,0


In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.94
Test took: 0:00:06


In [ ]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [ ]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [ ]:
import pandas as pd
text = pd.read_csv('/content/real_final_2.csv', encoding = 'utf-8')

In [ ]:
logits = test_sentences(['아 벌써 50명이야?'])
np.argmax(logits)

[[-4.145627 -1.908369  4.959878]]
2


## 모델 저장

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Bert_긍우중_epoch_10.pth')

In [ ]:
torch.save(model, '/content/drive/MyDrive/Bert_긍우중_all_epoch_10.pth')

In [ ]:
model.load_state_dict(torch.load('/content/gdrive/MyDrive/공공빅데이터_감정/긍분중.pth'))

<All keys matched successfully>

In [ ]:
model = torch.load('/content/drive/MyDrive/Bert_긍우중_all_epoch_10.pth')

In [ ]:
logits = test_sentences(['곧 좋아질거야'])

print(logits)
print(np.argmax(logits))

[[-1.8881178  0.9006984  1.0844351]]
2


In [ ]:
test.to_csv('/content/drive/MyDrive/test.csv', encoding = 'utf-8-sig')